# Aprendizem Profunda - LSTM
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [7]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [8]:
df = pd.read_csv("data/fixed_dataset_final.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [9]:
def to_csv(results_df, name):
    os.makedirs("score", exist_ok=True)
    path = f"score/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [10]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [11]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [12]:
# Tokenize the data
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [13]:
data_set_professor = pd.read_csv("data/dataset1_inputs.csv", sep='\t', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=200)  

### Early Stopping

In [14]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=3,            
    restore_best_weights=True,           
)

### Reduce_lr

In [15]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

## LSTM

In [16]:
model_lstm = Sequential([
    Embedding(input_dim=30000, output_dim=32),  

    LSTM(16, activation="tanh", return_sequences=True, 
         kernel_regularizer=l2(0.005), recurrent_dropout=0.5), 
    Dropout(0.5),

    LSTM(8, kernel_regularizer=l2(0.005), recurrent_dropout=0.5, return_sequences=False),
    Dropout(0.5),

    BatchNormalization(),

    Dense(16, activation="relu", kernel_regularizer=l2(0.003)), 
    Dropout(0.7),  

    Dense(1, activation="sigmoid")
])

optimizer = Adam(learning_rate=0.0001, clipnorm=1.0)
model_lstm.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

model_lstm.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,  
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)   

Epoch 1/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.5031 - loss: 1.0884 - val_accuracy: 0.5262 - val_loss: 1.0352 - learning_rate: 1.0000e-04
Epoch 2/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - accuracy: 0.5059 - loss: 1.0409 - val_accuracy: 0.5664 - val_loss: 1.0147 - learning_rate: 1.0000e-04
Epoch 3/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 9s 110ms/step - accuracy: 0.5147 - loss: 1.0217 - val_accuracy: 0.5864 - val_loss: 0.9956 - learning_rate: 1.0000e-04
Epoch 4/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 104ms/step - accuracy: 0.5425 - loss: 0.9949 - val_accuracy: 0.6343 - val_loss: 0.9766 - learning_rate: 1.0000e-04
Epoch 5/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - accuracy: 0.5503 - loss: 0.9667 - val_accuracy: 0.6898 - val_loss: 0.9568 - learning_rate: 1.0000e-04
Epoch 6/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 9s 106ms/step - accuracy: 0.5618 - loss: 0.9365 - val_accuracy: 0.7608 - val_loss: 0.9341 - learning_rate: 1.0000e-04
Epoch 7/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - accuracy: 0

**Test Set**

In [17]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_lstm, X_test, test_ids, "lstm_dataset1")
model_lstm.save("models/lstm.keras")

26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step


**Avaliação**

In [21]:

predict_and_save(model_lstm, X_professor, data_set_professor["ID"], "lstm_dataset1")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


In [ ]:
# 1. Carregar os arquivos
preds_df = pd.read_csv("score/lstm_dataset1.csv", sep='\t', on_bad_lines='skip')
labels_df = pd.read_csv("data/dataset1_outputs.csv", sep='\t', on_bad_lines='skip')

# 2. Garantir que as colunas corretas estão sendo usadas
# Assumindo que a coluna de previsões se chama 'Pred' e a de verdade se chama 'Label'
y_pred = preds_df['Label']
y_true = labels_df['Label']

# 3. Comparar previsões com os rótulos reais
print("Acurácia:", accuracy_score(y_true, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

Acurácia: 0.7333333333333333
Relatório de Classificação:
              precision    recall  f1-score   support

          AI       1.00      0.47      0.64        15
       Human       0.65      1.00      0.79        15

    accuracy                           0.73        30
   macro avg       0.83      0.73      0.71        30
weighted avg       0.83      0.73      0.71        30



## Classification Reports

In [20]:
print(classification_report(y_test, model_lstm.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step
              precision    recall  f1-score   support

         0.0       0.97      0.92      0.95       420
         1.0       0.92      0.97      0.95       391

    accuracy                           0.95       811
   macro avg       0.95      0.95      0.95       811
weighted avg       0.95      0.95      0.95       811

